In [7]:
import pandas as pd

max_length= 128 # 186


file_path = '/home/sunhuaikuan/ondemand/blue_papers/DNA_LLM_REVIEW/datasets/task05-methylation/GSM6637962_CpG_coverage20_GRCh38.bed.gz'     
df = pd.read_csv(file_path, sep='\t')
df['ROWID'] = df.index


df['CHROM'] = df['CHROM'].str.replace('chr', '', regex=False)
df=df[~df['CHROM'].str.contains('KI',na=False)]
df=df[~df['CHROM'].str.contains('GL',na=False)]
df=df[~df['CHROM'].str.contains('M',na=False)]


df['SIZE']=max_length

df['START']= df['FROM']-round(max_length / 2)
df['END']= df['START'] + df['SIZE']

df = df.rename(columns={'Percentage':'y'})
df = df.drop(['FROM','TO','Coverage'], axis=1)

df_origin = df
df_origin

,CHROM,y,ROWID,SIZE,START,END
0,1,100,0,128,788343,788471
1,1,80,1,128,788408,788536
2,1,95,2,128,788761,788889
3,1,90,3,128,788780,788908
4,1,100,4,128,788795,788923
...,...,...,...,...,...,...
54789,Y,65,54789,128,56870839,56870967
54790,Y,17,54790,128,56870919,56871047
54791,Y,34,54791,128,56871045,56871173
54792,Y,0,54792,128,56871438,56871566


In [2]:
# first_column = dataframe.iloc[:,0].values
# second_column = dataframe.iloc[:, 1].values
# third_column = dataframe.iloc[:, 2].values
# fifth_column = dataframe.iloc[:, 5].values

,sequence,y
0,GCAAAAAAAGCAAAACTCTGAAGAAAGAGAGAGAGAGGGAGAGAGG...,100
1,GATGAGAGACAGAGAGAAGGAGAGAGAAAGTACAAAAGAACGAATG...,80
2,GAATGGAATGGAATGGAATGGAATGGAATGGAATGGAATGGAATGG...,95
3,GGAATGGAATGGAATGGAATGGAATGGAGTGGAGTGGAATGGACCC...,90
4,GGAATGGAATGGAGTGGAGTGGAATGGACCCGAATGGAATGGAACG...,100
...,...,...
47948,GCCGGATCCCTTTCACTGACACATTAAAGTTAGAGCCACCCTAGGT...,65
47949,AGTGCTGGGTCTATCAGTCTCTCCCAACCATTCCCTGGGCTTTGCC...,17
47950,CTAAGTCAACTCCAGAGTTCACAGACCTCACTTCTGATCTCCACCA...,34
47951,CAGCCTACATTTTCCAGTCTTCCATTTACCTATTCAACTAATACCT...,0


### NOTICE: The following cell should run twice in the kernel to avoid total null result.

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Load pre-trained Caduceus model and tokenizer
model_name = "kuleshov-group/caduceus-ph_seqlen-131k_d_model-256_n_layer-16"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained(model_name, trust_remote_code=True)


# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

CaduceusForMaskedLM(
  (caduceus): Caduceus(
    (backbone): CaduceusMixerModel(
      (embeddings): CaduceusEmbeddings(
        (word_embeddings): Embedding(16, 256)
      )
      (layers): ModuleList(
        (0-15): 16 x Block(
          (mixer): BiMambaWrapper(
            (mamba_fwd): Mamba(
              (in_proj): Linear(in_features=256, out_features=1024, bias=False)
              (conv1d): Conv1d(512, 512, kernel_size=(4,), stride=(1,), padding=(3,), groups=512)
              (act): SiLU()
              (x_proj): Linear(in_features=512, out_features=48, bias=False)
              (dt_proj): Linear(in_features=16, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=256, bias=False)
            )
            (mamba_rev): Mamba(
              (in_proj): Linear(in_features=256, out_features=1024, bias=False)
              (conv1d): Conv1d(512, 512, kernel_size=(4,), stride=(1,), padding=(3,), groups=512)
              (act): SiLU()
          

### Mathod One: compute embeddings one by one. (works but slow, takes 10+ min)

In [10]:
# import dask.dataframe as dd
# from dask.diagnostics import ProgressBar
# import numpy as np

# # 设置并行参数和批处理大小
# num_parallel = 10

# # 创建一个DataFrame，包含DNA序列
# segments_df = pd.DataFrame(segments, columns=['sequence'])

# # 使用Dask对DataFrame进行并行化处理
# segments_ddf = dd.from_pandas(segments_df, npartitions=num_parallel)

# # 定义处理嵌入的函数
# def process_embedding(df):
#     embeddings = []
#     for dna in df['sequence']:
#         tokens = tokenizer(dna, return_tensors='pt', padding='max_length', max_length=512, truncation=True)
#         tokens = {key: val.to(device) for key, val in tokens.items()}

#         with torch.no_grad():
#             outputs = model(**tokens, output_hidden_states=True)
#             hidden_states = outputs.hidden_states
#             last_layer_embeddings = hidden_states[-1]  # 获取最后一层的嵌入
#             mean_embeddings = torch.mean(last_layer_embeddings, dim=1)  # 计算平均嵌入
#             mean_embeddings = mean_embeddings.view(mean_embeddings.shape[0], -1)
#             embeddings.append(mean_embeddings.cpu().numpy())

#     # 将嵌入结果转换为DataFrame
#     embeddings = np.vstack(embeddings)
#     return embeddings

# # 显示进度条并进行并行计算
# with ProgressBar():
#     ddf_embeddings = segments_ddf.map_partitions(process_embedding).compute()

# df_embeddings = pd.DataFrame(ddf_embeddings)
# df_embeddings

### Method Two: compute in batches (fast take 1+ min)

In [13]:
# 创建一个DataFrame，包含DNA序列
# df = pd.DataFrame(segments, columns=['sequence'])
# df

In [16]:
# %%time

import dask.dataframe as dd
import pandas as pd
import jax.numpy as jnp
from dask.diagnostics import ProgressBar

# Vectorized tokenization function
def vectorized_tokenizer(subsequences):
    # Assuming tokenizer.batch_tokenize can handle a list of subsequences
    # tokens_ids = [b[1] for b in tokenizer.batch_tokenize(subsequences)]
    # return jnp.asarray(tokens_ids, dtype=jnp.int32)

    # Tokenize the batch of sequences
    tokens = tokenizer(subsequences, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Move tokens to GPU
    tokens = {key: val.to(device) for key, val in tokens.items()}
    return tokens

# Vectorized embedding function
def vectorized_embedding(tokens):
    # random_key = jax.random.PRNGKey(0)
    # # Assuming forward_fn.apply() can handle batches of tokens
    # outs = forward_fn.apply(parameters, random_key, tokens)
    # return outs["embeddings_20"][:, 0, :]


    # Forward pass to compute last layer embeddings for the batch
    with torch.no_grad():
        outputs = model(**tokens, output_hidden_states=True)  # Enable output of hidden states
        hidden_states = outputs.hidden_states  # Access all hidden states
        last_layer_embeddings = hidden_states[-1]  # Get the last layer embeddings (batch_size, seq_len, hidden_size)
    
    # Compute the mean of the last layer embeddings across the token (sequence) dimension for each sequence in the batch
    # Dimension 1 corresponds to the token/sequence length, so we compute the mean along this axis
    mean_embeddings = torch.mean(last_layer_embeddings, dim=1)  # (batch_size, hidden_size)
    
    # If needed, squeeze out any extra dimensions (though this shouldn't be necessary after mean calculation)
    mean_embeddings_squeezed = mean_embeddings.squeeze(dim=1)

    return mean_embeddings_squeezed

# Tokenization and embedding combined in a batch-wise function
def process_batch(subsequences):
    tokens = vectorized_tokenizer(subsequences)
    embeddings = vectorized_embedding(tokens)
    return embeddings

# Function to apply on each Dask partition
# def apply_get_embeddings_dask(df):
#     # subsequences = df['ref_forward_sequence'].values  # Get all subsequences in the batch
#     subsequences = df['ref_forward_sequence'].tolist() 
#     embeddings = process_batch(subsequences)  # Process in a vectorized manner
#     embeddings_cpu = embeddings.cpu().numpy()
#     df['embedding'] = list(embeddings_cpu)  # Assign embeddings back to the DataFrame
#     return df


def apply_get_embeddings_dask(df):
    # subsequences = df['ref_forward_sequence'].values  # Get all subsequences in the batch
    subsequences = df['sequence'].tolist() 
    embeddings = process_batch(subsequences)  # Process in a vectorized manner
    embeddings_cpu = embeddings.cpu().numpy()
    
    # df['embedding'] = list(embeddings_cpu)  # Assign embeddings back to the DataFrame
    df2 = pd.DataFrame(embeddings_cpu, columns=[f'{i+1}' for i in range(embeddings_cpu.shape[1])])
    df = pd.concat([df.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)
    return df

In [23]:
%%time

import numpy as np
import pandas as pd
import dask.dataframe as dd
from datasets import load_dataset, load_from_disk

%run preprocess_utility.py

# Define chunk size (number of rows per chunk)
# chunksize = 10000  # Adjust chunk size according to your memory capacity
# num_parallel = 10

#================dataset -> df================
# code for loading homo_sapiens_dataset/train
# save_path="homo_sapiens_dataset"
# mydatasets = load_from_disk(save_path)
# # print(mydatasets)
# train = mydatasets['train']
# df = train.to_pandas()


# chunkid=0
# typename="methylation"
# typename="missense"
typename="methylation"


# 创建一个DataFrame，包含DNA序列
# df = pd.DataFrame(segments, columns=['sequence'])

for chunkid in range(0,1):
    # save_path="methylation_multisets"
    # chunk_save_path = f"{save_path}/train_{chunkid}"
    # dataset_chunk = load_from_disk(chunk_save_path)
    # df = dataset_chunk.to_pandas()
    
    
    # code for loading pathgenicity_noncoding_multisets/combined_dataset
    # save_path = "pathgenicity_noncoding_multisets"
    # df = load_from_disk(save_path+"/combined_dataset")
    
    
    #================df -> df's chunks================
    # Assume df is your huge DataFrame
    chunk_size = 10000  # Define the number of rows per chunk
    num_parallel = 10
    
    num_chunks = int(np.ceil(len(df) / chunk_size))  # Calculate the number of chunks
    
    # Split the DataFrame into chunks using array_split
    chunks = np.array_split(df, num_chunks)
    
    # Initialize an empty list to store the processed chunks
    processed_chunks = []
    # num_parallel=10
    
    #================process each chunk with dask's ddf================
    # Iterate over each chunk
    for chunk in chunks:
        # Process each chunk
        # print(chunk.head())  # Example: print the first few rows of the chunk
    
        ddf = dd.from_pandas(chunk, npartitions=num_parallel)  # Adjust 'npartitions' based on resources
    
        # Create metadata for Dask to understand the structure of the DataFrame
        # meta = chunk.copy()
        # meta['embedding'] = object  # Tokens will be of type object (adjust type accordingly)



        # sample_subsequences = chunk['ref_forward_sequence'].tolist()
        # sample_embeddings = process_batch(sample_subsequences).cpu().numpy()  # Get sample embeddings
        # num_embedding_columns = sample_embeddings.shape[1]
        num_embedding_columns = 256
        
        # Define metadata for the new DataFrame structure
        # Drop the 'embedding' column since it no longer exists
        meta = chunk.copy()
        meta = meta.drop(columns=['embedding'], errors='ignore')  # Drop 'embedding' if it exists
        # Add new embedding columns to the metadata
        for i in range(num_embedding_columns):
            meta[f'{i+1}'] = float  # Adjust type as necessary, float is common for embeddings


        
    
        # Apply the function in parallel using Dask
        ddf = ddf.map_partitions(apply_get_embeddings_dask, meta=meta)
    
        # Compute the result with progress tracking
        with ProgressBar():
            processed_chunk = ddf.compute()
    
        # Append processed chunk to list
        processed_chunks.append(processed_chunk)

        
    # Concatenate all processed chunks into a final DataFrame
    final_df = pd.concat(processed_chunks, ignore_index=True)

    # final_df = final_df.drop(columns=['sequence','rowid'])
    # final_df = swapfirst2last(final_df)

    final_df.to_csv(typename+'_caduceus_embedding_'+str(chunkid)+'.csv', index=False)
    print(f"{typename}_caduceus_embedding_{chunkid}.csv is created.")

final_df

<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

[########################################] | 100% Completed | 5.65 sms


<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

[########################################] | 100% Completed | 5.64 sms


<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

...

<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

[########################################] | 100% Completed | 5.64 sms


<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

[########################################] | 100% Completed | 5.65 sms
methylation_caduceus_embedding_1.csv is created.
CPU times: user 1min 27s, sys: 2.64 s, total: 1min 30s
Wall time: 1min 35s


,sequence,y,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,256
0,GCAAAAAAAGCAAAACTCTGAAGAAAGAGAGAGAGAGGGAGAGAGG...,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GATGAGAGACAGAGAGAAGGAGAGAGAAAGTACAAAAGAACGAATG...,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GAATGGAATGGAATGGAATGGAATGGAATGGAATGGAATGGAATGG...,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GGAATGGAATGGAATGGAATGGAATGGAGTGGAGTGGAATGGACCC...,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GGAATGGAATGGAGTGGAGTGGAATGGACCCGAATGGAATGGAACG...,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47948,GCCGGATCCCTTTCACTGACACATTAAAGTTAGAGCCACCCTAGGT...,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47949,AGTGCTGGGTCTATCAGTCTCTCCCAACCATTCCCTGGGCTTTGCC...,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47950,CTAAGTCAACTCCAGAGTTCACAGACCTCACTTCTGATCTCCACCA...,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47951,CAGCCTACATTTTCCAGTCTTCCATTTACCTATTCAACTAATACCT...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df

,sequence,rowid,y
0,GCAAAAAAAGCAAAACTCTGAAGAAAGAGAGAGAGAGGGAGAGAGG...,0,100
1,GATGAGAGACAGAGAGAAGGAGAGAGAAAGTACAAAAGAACGAATG...,1,80
2,GAATGGAATGGAATGGAATGGAATGGAATGGAATGGAATGGAATGG...,2,95
3,GGAATGGAATGGAATGGAATGGAATGGAGTGGAGTGGAATGGACCC...,3,90
4,GGAATGGAATGGAGTGGAGTGGAATGGACCCGAATGGAATGGAACG...,4,100
...,...,...,...
47948,GCCGGATCCCTTTCACTGACACATTAAAGTTAGAGCCACCCTAGGT...,54789,65
47949,AGTGCTGGGTCTATCAGTCTCTCCCAACCATTCCCTGGGCTTTGCC...,54790,17
47950,CTAAGTCAACTCCAGAGTTCACAGACCTCACTTCTGATCTCCACCA...,54791,34
47951,CAGCCTACATTTTCCAGTCTTCCATTTACCTATTCAACTAATACCT...,54792,0
